In [27]:
import pickle
import pathlib

import numpy as np
import pandas as pd

In [28]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\marce\Desktop\Machine learning\ames_AndreVictor\data


In [29]:
processed_file_path = DATA_DIR / 'processed' / 'ames_with_correct_types.pkl'

with open(processed_file_path, 'rb') as file:
    (
        data,
        continuous_variables,
        discrete_variables,
        ordinal_variables,
        categorical_variables,
    ) = pickle.load(file)

In [30]:
numerical_data = data.select_dtypes('number').drop(columns='SalePrice').copy()
target = data['SalePrice'].copy()

In [31]:
from scipy.stats import anderson
for column, series in numerical_data.items():
    # Calculate correlation between the two columns
    corr = series.corr(target)

    series = series.loc[series != 0]
    log_series = series.apply(np.log)
    corr_log = log_series.corr(target)
    if abs(corr_log) > (abs(corr) + 0.05):
        print("Correlation between", column, "and the target is", corr)
        print("Correlation between log("+column+") and the target is", corr_log)
        print()
for col in continuous_variables:
    non_zero_values = data[data[col] != 0]

    anderson_stat, anderson_critical_values, anderson_significance_levels = anderson(np.log(non_zero_values[col]))

    if anderson_stat < anderson_critical_values[2]:  # Using a 5% significance level
        print(f"The data in {col} follows a lognormal distribution.")

Correlation between Lot.Area and the target is 0.2665492203284296
Correlation between log(Lot.Area) and the target is 0.3650759920066496

Correlation between BsmtFin.SF.2 and the target is 0.005891397783530734
Correlation between log(BsmtFin.SF.2) and the target is 0.1617833105405112

Correlation between X2nd.Flr.SF and the target is 0.26937335658959116
Correlation between log(X2nd.Flr.SF) and the target is 0.6085308920396726

Correlation between Low.Qual.Fin.SF and the target is -0.03765976523775168
Correlation between log(Low.Qual.Fin.SF) and the target is 0.1830498247176396

Correlation between Enclosed.Porch and the target is -0.12878744150780538
Correlation between log(Enclosed.Porch) and the target is 0.17917947313732152

Correlation between X3Ssn.Porch and the target is 0.03222464935882309
Correlation between log(X3Ssn.Porch) and the target is 0.15182811813431807

Correlation between Screen.Porch and the target is 0.11215121356053082
Correlation between log(Screen.Porch) and the

In [32]:
for column, series in numerical_data.items():
    # Calculate correlation between the two columns
    corr = series.corr(target)

    series = series.loc[series != 0]
    log_series = series.apply(np.log)
    corr_log = log_series.corr(target)
    if abs(corr_log) > (abs(corr) + 0.05):
        print("Correlation between", column, "and the target is", corr)
        print("Correlation between log("+column+") and the target is", corr_log)
        print()

Correlation between Lot.Area and the target is 0.2665492203284296
Correlation between log(Lot.Area) and the target is 0.3650759920066496

Correlation between BsmtFin.SF.2 and the target is 0.005891397783530734
Correlation between log(BsmtFin.SF.2) and the target is 0.1617833105405112

Correlation between X2nd.Flr.SF and the target is 0.26937335658959116
Correlation between log(X2nd.Flr.SF) and the target is 0.6085308920396726

Correlation between Low.Qual.Fin.SF and the target is -0.03765976523775168
Correlation between log(Low.Qual.Fin.SF) and the target is 0.1830498247176396

Correlation between Enclosed.Porch and the target is -0.12878744150780538
Correlation between log(Enclosed.Porch) and the target is 0.17917947313732152

Correlation between X3Ssn.Porch and the target is 0.03222464935882309
Correlation between log(X3Ssn.Porch) and the target is 0.15182811813431807

Correlation between Screen.Porch and the target is 0.11215121356053082
Correlation between log(Screen.Porch) and the